In [1]:
import requests
from bs4 import BeautifulSoup as Bs

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics
import numpy as np
from datetime import datetime as dt
import re
from math import nan
import requests
from bs4 import BeautifulSoup as Bs
from fake_useragent import UserAgent
import seaborn as sns
import matplotlib.pyplot as plt
import geopy.distance
import geopy
from geopy.geocoders import Nominatim
from multiprocessing.dummy import Pool as ThreadPool
UserAgent().chrome

'Mozilla/5.0 (Windows NT 4.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/37.0.2049.0 Safari/537.36'

In [3]:
def parsing_words(item):
    words_list = []
    pattern = re.compile('[a-zA-Z]+')
    for i in item:
        find_pattern = pattern.findall(i.text)
        if len(find_pattern)>0:
            words_list.append(pattern.findall(i.text)[0])
    return words_list


def get_html(url):
    response = requests.get(url, headers={'User-Agent': UserAgent().chrome})
    return response.text

In [4]:
url = 'https://s-english.ru/grammatika/adjective/anglijskie-prilagatelnye-s-perevodom'# ссылка на ресурс с хорошими прилагательными
url2 = 'https://akyla.net/anglijskaja-leksika/52-anglijskaja-leksika/5204-plokhie-prilagatelnye-na-anglijskom-s-perevodom'# с негативными
url3 = 'https://vk.com/wall-19511382_11390'# с негативными
url4 = 'https://lingvister.ru/blog/plohie-slova-v-angliyskom-yazyke-ili-this-damnable-english-vocabulary'# с негативными
url5 = 'https://magoosh.com/english-speaking/positive-adjectives-the-ultimate-list/'# с положительнымы
url6 = 'https://grammar.yourdictionary.com/parts-of-speech/adjectives/list-of-positive-adjectives.html'# с положительнымы

In [5]:
#Создаем списки с негативными прилагательными

bad_words4 = 'boring noisy petty hideous revolting repulsive stingy obnoxious dysfunctional spoiled slimy callous dreary Terrible Disgusting Revolting Unpalatable Sugary Unsavory Distasteful Sickening Gross Nasty Smelly Fetid Malodorous Rammish Ranckness Specific Hard Rough Dry Greasy Slimy Viscous Gooey Bland bored mediocre disgusting horrible inedible unhealthy junk spoiled rotten processed tasteless oily fatty stinking smelly greasy'.split(' ')


page2 = Bs(get_html(url2),'html.parser')
table_bad = page2.find('tbody').find_all('td')
bad_words = parsing_words(table_bad)


pattern = re.compile('[a-zA-Z]+')
page3 = Bs(get_html(url3),'html.parser')
bad_words2 = pattern.findall(page3.text)[10:-4]


page4 = Bs(get_html(url4),'html.parser')
bad_words3 = parsing_words(page4.find_all('strong'))

bad_words = bad_words+bad_words2+bad_words3+bad_words4

for k,i in enumerate(bad_words):
    bad_words[k] = i.lower()
    
bad_words = set(bad_words)

In [6]:
#Создаем списки с положительными прилагательными

good_words3 = 'best good well fun Enjoyable Warm welcoming Delicious Amazing Excellent Clean \
efficient great relaxed healthy fast hot fab Brilliant Fantastic tasty fresh tasty scrumptious \
wonderful luscious mouth-watering tempting decent unappetizing gourmet fine nutritious savoury \
sweet rich fragrant safety savory appetizing toothsome dainty flavorous gustable delicious \
scrumptious mouth-watering Fragrant Pleasant Aromatic Sweet Tender Juicy Creamy Crumbly Moist Spongy'.split(' ')

page = Bs(get_html(url),'html.parser')
table_good = page.find('table').find_all('p')
good_words = parsing_words(table_good)


good_words2 = []
page5 = Bs(get_html(url5),'html.parser')
for i in page5.find_all('li'):
    if i.find('strong')!=None:
        good_words2.append(i.find('strong').text)

        
good_words4 =  []  
page6 = Bs(get_html(url6),'html.parser')
for i in page6.find_all('section', class_="article-body"):
    for k in i.find_all('li'):
        good_words4.append(k.text)
        
        
good_words = good_words2+good_words3+good_words4

for k,i in enumerate(good_words):
    good_words[k] = i.lower()
    
good_words = set(good_words)

In [7]:
# Считываем негативные слова с текстового документа
new_list = []
f = open('negative_words.txt', 'r')
for line in f:
    line.replace(' ','')
    new_list.append(line[:-1])

In [8]:
new_list

['2-faced',
 '2-faces',
 'abnormal',
 'abolish',
 'abominable',
 'abominably',
 'abominate',
 'abomination',
 'abort',
 'aborted',
 'aborts',
 'abrade',
 'abrasive',
 'abrupt',
 'abruptly',
 'abscond',
 'absence',
 'absent-minded',
 'absentee',
 'absurd',
 'absurdity',
 'absurdly',
 'absurdness',
 'abuse',
 'abused',
 'abuses',
 'abusive',
 'abysmal',
 'abysmally',
 'abyss',
 'accidental',
 'accost',
 'accursed',
 'accusation',
 'accusations',
 'accuse',
 'accuses',
 'accusing',
 'accusingly',
 'acerbate',
 'acerbic',
 'acerbically',
 'ache',
 'ached',
 'aches',
 'achey',
 'aching',
 'acrid',
 'acridly',
 'acridness',
 'acrimonious',
 'acrimoniously',
 'acrimony',
 'adamant',
 'adamantly',
 'addict',
 'addicted',
 'addicting',
 'addicts',
 'admonish',
 'admonisher',
 'admonishingly',
 'admonishment',
 'admonition',
 'adulterate',
 'adulterated',
 'adulteration',
 'adulterier',
 'adversarial',
 'adversary',
 'adverse',
 'adversity',
 'afflict',
 'affliction',
 'afflictive',
 'affront',


In [9]:
# Добавляем их в bad_words
bad_words= list(set(list(bad_words) + new_list))

In [10]:
# Считываем положительные слова с текстового документа
new_list = []
f = open('positive_words.txt', 'r')
for line in f:
    line.replace(' ','')
    new_list.append(line[:-1])

In [11]:
# Добавляем их в bad_words
good_words= list(set(list(good_words) + new_list))

Запишем результат в тектовый документ

In [12]:
f = open('end_neg_words.txt', 'w')
for word in bad_words:
    f.write(word + '\n')
f.close()

f = open('end_pos_words.txt', 'w')
for word in good_words:
    f.write(word + '\n')
f.close()